In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import torch, os, re, pandas as pd, json
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig

import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
from torch import optim
import gc
from torch.nn import DataParallel

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
device = "cuda:4" if torch.cuda.is_available() else "cpu"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')

In [5]:
# head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(device)
head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl')

In [6]:
import os

pos_path = "/home/ubuntu/IMDB_train/pos/"
neg_path = "/home/ubuntu/IMDB_train/neg/"


pos_token_list = []
pos_text_list = []
neg_token_list = []
neg_text_list = []

os.chdir(pos_path)
  

for file in os.listdir():
    if not file.endswith('.txt'):
        continue
    with open(pos_path+file, 'r') as f:
        pos_text = f.read()
        # put into text list
        pos_text_list.append(pos_text)
        #tokenize and put into token list
        #pos_token_list.append(base_tokenizer.encode(pos_text, return_tensors = 'pt'))

os.chdir(neg_path)

for file in os.listdir():
    if not file.endswith('.txt'):
        continue
    with open(neg_path+file, 'r') as f:
        neg_text = f.read()
        # put into text list
        neg_text_list.append(neg_text)
        #tokenize and put into token list
        #neg_token_list.append(base_tokenizer.encode(neg_text_list, return_tensors = 'pt'))

In [7]:
from io import StringIO
from html.parser import HTMLParser

# ref https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()
# cleaned = test_text_pos.replace('<br /><br />', ' ')
# print(cleaned)

def clean_imdb(review_list):
    for review in range(len(review_list)):
        cleaned_review = strip_tags(review_list[review])
        review_list[review] = cleaned_review

In [8]:
clean_imdb(neg_text_list)

In [9]:
# df_pos_tokens = pd.DataFrame(pos_token_list, columns = ['tokens'])
# print(df_pos_tokens)
df_neg_text = pd.DataFrame(neg_text_list, columns = ['text'])
print(df_neg_text)

                                                    text
0      the tortuous emotional impact is degrading, wh...
1      The Horror Channel plays nothing but erotic so...
2      I saw this piece of garbage on AMC last night,...
3      The film begins with promise, but lingers too ...
4      the more i think about it, there was nothing r...
...                                                  ...
12495  There's a lot of good that can be said for thi...
12496  Wow. Some movies just leave me speechless. Thi...
12497  This is only a response to the yahoo who says ...
12498  This film is about a family trying to come to ...
12499  Kojak meets the mafia. Telly Savales is one of...

[12500 rows x 1 columns]


In [10]:
# text = "I work as a data scientist"
# text_ids = base_tokenizer.encode(text, return_tensors = 'pt')
# print(text_ids.shape)
# print(head_model(text_ids.to(device)).logits.shape)

In [11]:
class ImdbNeg(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2-xl", max_length=1024):

        self.tokenizer = base_tokenizer # can change
        self.text = []

        for row in df_neg_text['text']:
            self.text.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.text = self.text[:20000]
        self.text_count = len(self.text)
        
    def __len__(self):
        return self.text_count

    def __getitem__(self, item):
        return self.text[item]
    
dataset = ImdbNeg(df_neg_text['text'], truncate=True, gpt2_type="gpt2-xl")  

In [12]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [13]:
# def train(
#     dataset, model, tokenizer,
#     batch_size=10, epochs=20, lr=2e-6,
#     max_seq_len=1000, warmup_steps=200,
#     gpt2_type="gpt2-xl", output_dir=".", output_prefix="wreckgar",
#     test_mode=False,save_model_on_epoch=False,
# ):
#     acc_steps = 100
#     # device=torch.device("cuda")
#     model = model.to(device)
#     model.train()
#     t = torch.cuda.get_device_properties(1).total_memory
#     r = torch.cuda.memory_reserved(1)
#     a = torch.cuda.memory_allocated(1)
#     f = r-a  # free inside reserved
#     print(t/1000000000)
#     print(r/1000000000)
#     print(a/1000000000)
#     print(f/1000000000)
    
#     optimizer = optim.RAdam(model.parameters(), lr=lr)
#     # scheduler = get_linear_schedule_with_warmup(
#     #     optimizer, num_warmup_steps=warmup_steps, num_training_steps=1250*20
#     # )

#     train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#     loss=0
#     accumulating_batch_count = 0
#     input_tensor = None

#     for epoch in range(epochs):
#         count = 0
#         print(f"Training epoch {epoch}")
#         print(loss)
#         for idx, entry in tqdm(enumerate(train_dataloader)):
#             #print(len(train_dataloader))
#             (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)
#             del entry
#             del remainder
#             gc.collect()
#             if count%10 == 0:
#                 print("FIRST STEP BELOW")
#                 t = torch.cuda.get_device_properties(1).total_memory
#                 r = torch.cuda.memory_reserved(1)
#                 a = torch.cuda.memory_allocated(1)
#                 f = r-a  # free inside reserved
#                 print(t/1000000000)
#                 print(r/1000000000)
#                 print(a/1000000000)
#                 print(f/1000000000)
#             if carry_on and idx != len(train_dataloader) - 1:
#                 continue

#             # input_tensor = input_tensor.to(device)
#             outputs = model(input_tensor.to(device), labels=input_tensor.to(device))
#             print(outputs)
#             loss = outputs[0]
#             loss.backward()
#             del outputs
#             del loss #?
#             gc.collect()
#             # torch.cuda.empty_cache()
#             if count%10 == 0:
#                 print("AFTER LOSS BACK BELOW")
#                 t = torch.cuda.get_device_properties(1).total_memory
#                 r = torch.cuda.memory_reserved(1)
#                 a = torch.cuda.memory_allocated(1)
#                 f = r-a  # free inside reserved
#                 print(t/1000000000)
#                 print(r/1000000000)
#                 print(a/1000000000)
#                 print(f/1000000000)

#             if (accumulating_batch_count % batch_size) == 0:
#                 optimizer.step()
#                 # scheduler.step()
#                 optimizer.zero_grad(set_to_none=True)
#                 model.zero_grad(set_to_none=True)
            
#             if count%10 == 0:
#                 print("AFTER MODEL STEP BELOW")
#                 t = torch.cuda.get_device_properties(1).total_memory
#                 r = torch.cuda.memory_reserved(1)
#                 a = torch.cuda.memory_allocated(1)
#                 f = r-a  # free inside reserved
#                 print(t/1000000000)
#                 print(r/1000000000)
#                 print(a/1000000000)
#                 print(f/1000000000)
#             accumulating_batch_count += 1
#             del input_tensor
#             # del outputs
#             # del loss #?
#             gc.collect()
#             # torch.cuda.empty_cache()
#             input_tensor = None
#             if count%10 == 0:
#                 print("END EXAMPLE BELOW")
#                 t = torch.cuda.get_device_properties(1).total_memory
#                 r = torch.cuda.memory_reserved(1)
#                 a = torch.cuda.memory_allocated(1)
#                 f = r-a  # free inside reserved
#                 print(t/1000000000)
#                 print(r/1000000000)
#                 print(a/1000000000)
#                 print(f/1000000000)
#             count+=1
#         if save_model_on_epoch:
#             torch.save(
#                 model.state_dict(),
#                 os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
#             )
#     return model

In [14]:
# def train(
#     dataset, model, tokenizer,
#     batch_size=10, epochs=20, lr=2e-6,
#     max_seq_len=1000, warmup_steps=200,
#     gpt2_type="gpt2-xl", output_dir=".", output_prefix="wreckgar",
#     test_mode=False,save_model_on_epoch=False,
# ):
#     acc_steps = 100
#     # device=torch.device("cuda")
    
#     model = DataParallel(model, device_ids = [4,5,6,7])
#     model = model.to(device)
#     model.train()

#     optimizer = optim.RAdam(model.parameters(), lr=lr)
#     # scheduler = get_linear_schedule_with_warmup(
#     #     optimizer, num_warmup_steps=warmup_steps, num_training_steps=1250*20
#     # )

#     train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#     loss=0
#     accumulating_batch_count = 0
#     input_tensor = None

#     for epoch in range(epochs):

#         print(f"Training epoch {epoch}")
#         print(loss)
#         for idx, entry in tqdm(enumerate(train_dataloader)):
#             #print(len(train_dataloader))
#             (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)
#             del entry
#             del remainder
#             gc.collect()
#             if carry_on and idx != len(train_dataloader) - 1:
#                 continue

#             input_tensor = input_tensor.to(device)
#             outputs = model(input_tensor, labels=input_tensor)
#             loss = outputs[0]
#             loss.backward()
#             del input_tensor
#             del outputs
#             del loss #?
#             gc.collect()
#             if (accumulating_batch_count % batch_size) == 0:
#                 optimizer.step()
#                 # scheduler.step()
#                 optimizer.zero_grad(set_to_none=True)
#                 model.zero_grad(set_to_none=True)

#             accumulating_batch_count += 1
#             # del input_tensor
#             # del outputs
#             # del loss #?
#             gc.collect()
#             input_tensor = None
#         if save_model_on_epoch:
#             torch.save(
#                 model.state_dict(),
#                 os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
#             )
#     return model

In [15]:
def pad_collate(batch):
    max_len = float("-inf")
    stacked_batch = None
    for ten in batch:
        if ten.size()[0] > max_len:
            max_len = ten.size()[0]
    for ten in batch:
        cur_ten_len = ten.size()[0]
        # running_ten = ten
        for cur_ten_idx in range(cur_ten_len, max_len):
            # print(type(ten))
            ten = torch.cat((ten ,torch.tensor(base_tokenizer.encode(base_tokenizer.eos_token))))
        if stacked_batch is None:
            stacked_batch = ten.unsqueeze(0)
        else:
            stacked_batch = torch.cat((stacked_batch, ten.unsqueeze(0)))
            
    return stacked_batch

In [16]:
def train(
    dataset, model, tokenizer,
    batch_size=10, epochs=20, lr=2e-5,
    max_seq_len=1000, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    # device=torch.device("cuda")
    #model = DataParallel(model, device_ids = [3])
    model = model.to(device)
    model.train()

    optimizer = optim.RAdam(model.parameters(), lr=lr)
    # scheduler = get_linear_schedule_with_warmup(
    #     optimizer, num_warmup_steps=warmup_steps, num_training_steps=1250*20
    # )

    train_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=pad_collate, shuffle=True)

    # train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        # print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            
            #print(len(train_dataloader))
            # print(f"0 {entry}")
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)
            # print(f"1 {input_tensor}")
            del entry
            # print(f"2 {input_tensor}")
            del remainder
            # print(f"3 {input_tensor}")
            gc.collect()
            # print("before")
            # print(carry_on)
            # print(idx)
            # print(len(train_dataloader) - 1)
            # print(f"4 {input_tensor}")
            if carry_on and idx != len(train_dataloader) - 1:
                continue
            # print("after")
            # print(f"5 {input_tensor}")
            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            if idx%100:
                print(f"LOSS: {loss}")
            loss.backward()
            del input_tensor
            del outputs
            del loss #?
            gc.collect()
            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                # scheduler.step()
                optimizer.zero_grad(set_to_none=True)
                model.zero_grad(set_to_none=True)

            accumulating_batch_count += 1
            # del input_tensor
            # del outputs
            # del loss #?
            gc.collect()
            input_tensor = None
        loss = 0
        input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [17]:
# # pos_imdb_model = head_model
torch.cuda.empty_cache()
t = torch.cuda.get_device_properties(1).total_memory
r = torch.cuda.memory_reserved(1)
a = torch.cuda.memory_allocated(1)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.358472704
0.0
0.0
0.0


In [18]:
neg_imdb_model = train(dataset, head_model, base_tokenizer, epochs = 5, max_seq_len=1000, batch_size=1)

Training epoch 0


1it [00:00,  3.93it/s]

LOSS: 3.221679925918579


3it [00:02,  1.31it/s]

LOSS: 2.783567190170288


...

LOSS: 0.6070614457130432


2256it [23:33,  1.74it/s]

LOSS: 1.0746431350708008


2258it [23:34,  1.60it/s]


KeyboardInterrupt: 

In [19]:
torch.save(head_model.state_dict(), "/home/ubuntu/neg_imdb_model_intial_more_complex")

KeyboardInterrupt: 

In [ ]:
# torch.save(neg_imdb_model.state_dict(), "/home/ubuntu/weird_neg_imdb_model_saved_dif")

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)